Obtenemos los datos desde el datasets

In [22]:
import pandas as pd

# Cargar datos desde un archivo CSV

df = pd.read_csv("datos.csv")

print(df.columns)
df.head()

Index(['time', 'weather_code (wmo code)', 'temperature_2m_max (°C)',
       'temperature_2m_min (°C)', 'precipitation_sum (mm)',
       'wind_speed_10m_max (km/h)', 'wind_gusts_10m_max (km/h)',
       'sunshine_duration (s)', 'precipitation_hours (h)'],
      dtype='object')


,time,weather_code (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),wind_speed_10m_max (km/h),wind_gusts_10m_max (km/h),sunshine_duration (s),precipitation_hours (h)
0,2010-01-01,73,-1.6,-3.2,2.9,16.3,31.0,0.00,12.0
1,2010-01-02,71,-1.2,-5.7,1.0,15.8,23.0,2114.82,8.0
2,2010-01-03,71,-4.9,-8.8,0.0,7.1,16.6,12470.87,0.0
3,2010-01-04,3,-6.3,-9.5,0.0,13.0,23.8,0.00,0.0
4,2010-01-05,3,-5.4,-11.7,0.0,11.2,20.5,16419.52,0.0


Transformamos los datos de lluvia a 1 si llueve y 0 si no llueve

In [23]:

def clasificar_lluvia(mm):
    if mm == 0:
        return 0  # Sin lluvia
    elif mm < 5:
        return 1  # Lluvia ligera
    elif mm < 20:
        return 2  # Lluvia moderada
    else:
        return 3  # Lluvia intensa

# Aplicar función
df["lluvia_clase"] = df["precipitation_sum (mm)"].apply(clasificar_lluvia)

# Selección de atributos predictivos
X = df[[
    "weather_code (wmo code)",
    "temperature_2m_max (°C)",
    "temperature_2m_min (°C)",
    "wind_speed_10m_max (km/h)",
    "wind_gusts_10m_max (km/h)",
    "sunshine_duration (s)"
]]
y = df["lluvia_clase"]


df.head()

,time,weather_code (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),wind_speed_10m_max (km/h),wind_gusts_10m_max (km/h),sunshine_duration (s),precipitation_hours (h),lluvia_clase
0,2010-01-01,73,-1.6,-3.2,2.9,16.3,31.0,0.00,12.0,1
1,2010-01-02,71,-1.2,-5.7,1.0,15.8,23.0,2114.82,8.0,1
2,2010-01-03,71,-4.9,-8.8,0.0,7.1,16.6,12470.87,0.0,0
3,2010-01-04,3,-6.3,-9.5,0.0,13.0,23.8,0.00,0.0,0
4,2010-01-05,3,-5.4,-11.7,0.0,11.2,20.5,16419.52,0.0,0


Limpiamos los datos eliminando los datos nulos

In [24]:
# Por defecto el metodo dropna() elimina las filas que tienen al menos un valor NaN
df_limpio = df.dropna()

df_limpio

,time,weather_code (wmo code),temperature_2m_max (°C),temperature_2m_min (°C),precipitation_sum (mm),wind_speed_10m_max (km/h),wind_gusts_10m_max (km/h),sunshine_duration (s),precipitation_hours (h),lluvia_clase
0,2010-01-01,73,-1.6,-3.2,2.9,16.3,31.0,0.00,12.0,1
1,2010-01-02,71,-1.2,-5.7,1.0,15.8,23.0,2114.82,8.0,1
2,2010-01-03,71,-4.9,-8.8,0.0,7.1,16.6,12470.87,0.0,0
3,2010-01-04,3,-6.3,-9.5,0.0,13.0,23.8,0.00,0.0,0
4,2010-01-05,3,-5.4,-11.7,0.0,11.2,20.5,16419.52,0.0,0
...,...,...,...,...,...,...,...,...,...,...
3647,2019-12-27,51,4.0,0.2,0.2,16.3,28.8,7040.66,2.0,1
3648,2019-12-28,3,2.2,-2.6,0.0,12.2,23.4,20937.14,0.0,0
3649,2019-12-29,3,1.4,-3.6,0.0,13.5,23.8,21436.21,0.0,0
3650,2019-12-30,3,7.0,1.3,0.0,21.1,35.3,15624.59,0.0,0


Separamos los datos en datos de entrenamiento y en datos de prueba

In [25]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler


# División en entrenamiento y prueba con estratificación
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Escalar para modelos sensibles (SVM, LR)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


Prueba modelo


In [26]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("Random Forest:\n", classification_report(y_test, y_pred_rf))

Random Forest:
               precision    recall  f1-score   support

           0       1.00      0.98      0.99       333
           1       0.88      0.95      0.92       321
           2       0.67      0.56      0.61        72
           3       0.00      0.00      0.00         5

    accuracy                           0.92       731
   macro avg       0.64      0.62      0.63       731
weighted avg       0.91      0.92      0.91       731



c:\Users\datit\OneDrive\Documentos\ICC732-1 INGENIERIA DE DATOS\Prueba 3\taller-3\taller-3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\datit\OneDrive\Documentos\ICC732-1 INGENIERIA DE DATOS\Prueba 3\taller-3\taller-3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\datit\OneDrive\Documentos\ICC732-1 INGENIERIA DE DATOS\Prueba 3\taller-3\taller-3\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being s